In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor, Lambda
import matplotlib.pyplot as plt
from torch import nn
import torchvision.models as models
import torch.optim as optim
import torch.nn.functional as F
import os

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"Current device: {torch.cuda.get_device_name(device)}")
else:
    device = torch.device("cpu")
    print("No GPU available, using CPU instead.")
    
transform = transforms.Compose(
    [transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

In [ ]:
trainset = datasets.Flowers102(root = 'data', split= 'train', download = True, transform=transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True, num_workers=2)

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
        self.conv5 = nn.Conv2d(256, 512, 3, padding=1)
        self.conv6 = nn.Conv2d(512, 1024, 3, padding=1)
        self.fc1 = nn.Linear(1024 * 28 * 28, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 102)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv5(x))
        x = F.relu(self.conv6(x))
        x = F.max_pool2d(x, 2)
        x = x.view(-1, 1024 * 28 * 28)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
model = NeuralNetwork()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#model.to(device) # move the model to the device

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
def train_loop():
    for epoch in range(1):  # loop over the dataset multiple times
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            if i % 200 == 199:    # print every 200 mini-batches
                print('[%d, %5d] loss: %.3f' %
                      (epoch + 1, i + 1, running_loss / 200))
                running_loss = 0.0
        print("Epoch " + str(epoch+1) + "\n ")

    print('Finished Training')

In [ ]:
testset = datasets.Flowers102(root='data', split='test', download =True,transform=transform)

testloader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=True, num_workers=2)

In [ ]:
model.eval()

In [ ]:
def test_loop():
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in testloader:


            # make prediction
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)

            # calculate accuarcy
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        print('Accuracy on the test set: {:.2f}%'.format(100 * correct / total))

In [ ]:
train_loop()

In [ ]:
test_loop()